In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
from tqdm import tqdm
import os
from tqdm import tqdm

In [3]:
#data_raw = pd.read_excel('__file_location__/emo_training.xlsx')
data_raw.columns

Index(['번호', '연령', '성별', '상황키워드', '신체질환', '감정_대분류', '감정_소분류', '사람문장1',
       '시스템응답1', '사람문장2', '시스템응답2', '사람문장3', '시스템응답3', '사람문장4', '시스템응답4'],
      dtype='object')

In [4]:
data_train = data_raw.loc[:,'감정_대분류':'시스템응답4']
print(data_train.columns)

Index(['감정_대분류', '감정_소분류', '사람문장1', '시스템응답1', '사람문장2', '시스템응답2', '사람문장3',
       '시스템응답3', '사람문장4', '시스템응답4'],
      dtype='object')


In [5]:
data_train['감정_대분류'].unique()

array(['기쁨', '불안', '당황', '슬픔', '분노', '상처', '불안 ', '기쁨 '], dtype=object)

In [6]:
{index:mood for index, mood in enumerate(data_train['감정_대분류'].unique())}

{0: '기쁨', 1: '불안', 2: '당황', 3: '슬픔', 4: '분노', 5: '상처', 6: '불안 ', 7: '기쁨 '}

In [7]:
{index:mood for index, mood in enumerate(data_train['감정_소분류'].unique())}

{0: '신이 난',
 1: '스트레스 받는',
 2: '당황',
 3: '안도',
 4: '취약한',
 5: '우울한',
 6: '구역질 나는',
 7: '좌절한',
 8: '눈물이 나는',
 9: '배신당한',
 10: '남의 시선을 의식하는',
 11: '죄책감의',
 12: '당혹스러운',
 13: '분노',
 14: '두려운',
 15: '낙담한',
 16: '회의적인',
 17: '마비된',
 18: '혼란스러운',
 19: '흥분',
 20: '한심한',
 21: '염세적인',
 22: '안달하는',
 23: '걱정스러운',
 24: '조심스러운',
 25: '충격 받은',
 26: '상처',
 27: '불안',
 28: '노여워하는',
 29: '기쁨',
 30: '짜증내는',
 31: '환멸을 느끼는',
 32: '비통한',
 33: '혐오스러운',
 34: '억울한',
 35: '만족스러운',
 36: '부끄러운',
 37: '버려진',
 38: '고립된',
 39: '괴로워하는',
 40: '자신하는',
 41: '외로운',
 42: '방어적인',
 43: '편안한',
 44: '질투하는',
 45: '감사하는',
 46: '초조한',
 47: '실망한',
 48: '슬픔',
 49: '신뢰하는',
 50: '가난한, 불우한',
 51: '후회되는',
 52: '툴툴대는',
 53: '성가신',
 54: '열등감',
 55: '희생된',
 56: '느긋',
 57: '악의적인'}

In [8]:
def emo_eco(data):
    data_c = data.copy()
    data_c.columns = ['main_cate', 'sub_cate', 'usr1', 'sys1', 'usr2', 'sys2', 'usr3', 'sys3', 'usr4', 'sys4']

    data_c['mc_enco'] = data_c['main_cate']
    data_c['sub_enco'] = data_c['sub_cate']

    for i, mc in enumerate(data_c['main_cate'].unique()):
        data_c.loc[:,'mc_enco'][data_c.loc[:,'mc_enco'] == mc] = i

    for i, mc in enumerate(data_c['sub_cate'].unique()):
        data_c.loc[:,'sub_enco'][data_c.loc[:,'sub_enco'] == mc] = i

    data_c = data_c.loc[:,'usr1':'sub_enco']
    return data_c

data_enco = emo_eco(data_train)

In [9]:
def reshape_data(data):
    final = []
    talk = data.iloc[:, :-2]

    for i in tqdm(range(len(talk))):
        mc_enco = data.iloc[i, :]['mc_enco']
        sub_enco = data.iloc[i, :]['sub_enco']
        cvs = talk.iloc[i, :]

        for cvs_i in range(len(cvs)//2):
            if (pd.notnull(cvs['usr' + str(cvs_i + 1)])) and (pd.notnull(cvs['sys' + str(cvs_i + 1)])):
                usr = cvs['usr' + str(cvs_i + 1)]
                sys = cvs['sys' + str(cvs_i + 1)]
                final.append([usr, sys, mc_enco, sub_enco])
    df_final = pd.DataFrame(final)
    df_final.columns = ['usr', 'sys', 'mc_enco', 'sub_enco']
    return df_final

prepro_df = reshape_data(data_enco)

for c_name in ['usr', 'sys']:
    prepro_df[c_name] = prepro_df[c_name].apply(lambda x: x.replace('\xa0', ''))

#prepro_df.to_csv('/content/drive/MyDrive/Saltlux 인공지능 자연어처리/220500_Final_project/220508_1st_output/dataset/df_preprocessing_220524.csv', encoding='cp949', index = False)

100%|██████████| 40879/40879 [00:16<00:00, 2520.10it/s]


In [10]:
prepro_df.head()

,usr,sys,mc_enco,sub_enco
0,아내가 드디어 출산하게 되어서 정말 신이 나.,아내분이 출산을 하시는군요. 정말 축하드려요.,0,0
1,아 지금 정말 신이 나.,잘 된 일이네요.,0,0
2,아기가 점점 클게 벌써 기대가 되네. 내가 많이 놀아줘야지.,좋은 아빠가 되실 거 같아요. 진심으로 축하드려요.,0,0
3,당뇨랑 합병증 때문에 먹어야 할 약이 열 가지가 넘어가니까 스트레스야.,약 종류가 많아 번거로우시겠어요.,1,1
4,건강할 때 관리 좀 잘할걸 하는 생각이 들더라고.,현재 상황에서 변화를 주기 위해 어떻게 하면 될까요?,1,1


In [11]:
prepro_df_nq = prepro_df.copy()
for i, sys_ans in enumerate(prepro_df.loc[:, 'sys']):
  if sys_ans.endswith('?'):
    prepro_df_nq.drop(index = i, axis = 0, inplace = True)
  if (i % 1000) == 0:
    print(i)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000


In [12]:
prepro_df_nq.to_csv('/content/drive/MyDrive/Saltlux 인공지능 자연어처리/220500_Final_project/220508_1st_output/dataset/no_question_220525.csv', encoding='cp949', index = False)

In [13]:
emo_conv = pd.read_csv('/content/drive/MyDrive/Saltlux 인공지능 자연어처리/220500_Final_project/220511/emo_conv.csv', encoding = 'cp949')

In [14]:
emo_conv['mc_enco'] = '<unk_s>'
emo_conv['sub_enco'] = '<unk_s>'

In [15]:
emo_conv.head()

,usr,sys,mc_enco,sub_enco
0,제 감정이 이상해진 것 같아요. 남편만 보면 화가 치밀어 오르고 감정 조절이 안되요.,감정이 조절이 안 될 때만큼 힘들 때는 없는 거 같아요.,<unk_s>,<unk_s>
1,제 감정이 이상해진 것 같아요. 남편만 보면 화가 치밀어 오르고 감정 조절이 안되요.,저도 그 기분 이해해요. 많이 힘드시죠?,<unk_s>,<unk_s>
2,제 감정이 이상해진 것 같아요. 남편만 보면 화가 치밀어 오르고 감정 조절이 안되요.,"그럴 때는 밥은 잘 먹었는지, 잠은 잘 잤는지 체크해보는 것도 좋아요.",<unk_s>,<unk_s>
3,더 이상 내 감정을 내가 컨트롤 못 하겠어.,감정이 조절이 안 될 때만큼 힘들 때는 없는 거 같아요.,<unk_s>,<unk_s>
4,더 이상 내 감정을 내가 컨트롤 못 하겠어.,저도 그 기분 이해해요. 많이 힘드시죠?,<unk_s>,<unk_s>


In [16]:
data_unified = pd.concat([prepro_df_nq, emo_conv], axis = 0).reset_index(drop=True)

In [17]:
#data_unified.to_csv('/content/drive/MyDrive/Saltlux 인공지능 자연어처리/220500_Final_project/220525/unified.csv', encoding = 'cp949')

In [22]:
len(emo_conv)

27604

In [24]:
prepro_df_nq['sub_enco'].unique()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 29, 30, 31, 32, 33, 34, 35,
       36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 27, 46, 47, 48, 49, 50, 28,
       51, 52, 53, 54, 55, 56, 57])

In [26]:
len(emo_conv)/57

484.280701754386

In [36]:
data_equal = pd.DataFrame({})

In [37]:
#485개씩
for i in prepro_df_nq['sub_enco'].unique():
  downsample = prepro_df_nq[prepro_df_nq['sub_enco'] == i].sample(485)
  data_equal = pd.concat([data_equal, downsample], axis = 0)

In [42]:
data_equal

,usr,sys,mc_enco,sub_enco
88963,은퇴 후에 가족들끼리 시작한 음식점 사업이 승승장구 해서 너무 기분이 좋아!,은퇴 후에 가족들끼리 시작한 사업의 성공이 기쁘시군요.,0,0
43928,맞아. 고마우니까 자식들 놀러오면 맛있는 거라도 잔뜩 해줘야지.,자식들에게 고마움을 느끼고 계시는군요.,0,0
107781,앞으로 더 노력해서 좋은 성과 얻어야지.,앞으로도 좋은 소식만 있으시길 바라요.,0,0
95750,응. 나보다 부모님이 더 좋아하시는 것 같네.,부모님이 더 좋아하시는군요. 그동안 고생한 보람이 있네요.,0,0
5372,이번 주에 샌드위치 휴일이 있어서 너무 좋아.,주중에 쉬는 날이 있어서 좋으신가 봐요.,0,0
...,...,...,...,...
32535,맞아. 악의적이라고 해도 어쩔 수 없는 게 그 사람도 나한테 그랬잖아.,눈에는 눈 이에는 이 방식으로 동료에게 복수를 하시려는 것처럼 보여요.,4,57
15314,과장님이 짧은 기간 내에 일을 너무 많이 주셔서 그냥 대충 해버렸어. 짜증 나.,과장님이 과중한 업무를 주셔서 짜증이 나셨군요.,4,57
37366,어떤 친구가 참석 못 한 다른 친구에 대해 악담을 하는데 심하더라고. 그렇게 살면 ...,악담의 정도가 지나쳐서 더 기분이 안 좋으셨군요.,4,57
9404,김 씨가 나를 밀어서 빠뜨리려 했어. 몹시 화가 나.,김 씨가 밀어서 화가 나셨군요.,4,57


In [43]:
data_equal_uni = pd.concat([data_equal, emo_conv], axis =0).reset_index(drop=True)

In [45]:
#data_equal_uni.to_csv('__file_location__/data_equal_uni.csv', encoding = 'cp949')